In [337]:
# !pip install mysql-connector-python

In [338]:
import pandas as pd
import numpy as np
import re

from array import array

import mysql.connector

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Conexión a la BDD

In [339]:
cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="nAddLeo56.",
    database="formula1db"
)

In [340]:
cursor = cnx.cursor()
cursor.execute("SELECT * FROM f1_data")
rows = cursor.fetchall()

query = 'SELECT * FROM f1_data'

## Creación del dataset

In [341]:
df = pd.read_sql(query, cnx)
df.head(2)

C:\Users\user\AppData\Local\Temp\ipykernel_12476\838681342.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


,id,driver_number,driver_name,driver_abb,driver_total_seasons,driver_season_rank,driver_season_points,driver_season_gps,driver_season_wins,driver_season_podiums,...,circuit_country,circuit_name,circuit_length,circuit_temp,circuit_rain,quali_pos,quali_time,race_pos,circuit_id,team_id
0,1,1,M VERSTAPPEN,VER,9.0,1.0,364.0,15.0,12.0,14.0,...,Bahrain,Bahrain International Circuit,5412.0,27.4317,0.0,1,89.708,1,7.0,1.0
1,2,11,S PEREZ,PER,13.0,2.0,219.0,15.0,2.0,8.0,...,Bahrain,Bahrain International Circuit,5412.0,27.4317,0.0,2,89.846,2,7.0,1.0


In [342]:
#Modificaciones del dataset
# df['circuit_id'] = df.groupby(['circuit_name']).ngroup()
df['team_id'] = df.groupby(['team_name']).ngroup()

df2 = df # <-- de este data set se extrae la info de texto

In [343]:
df = df.drop(['driver_name', 'driver_abb', 'team_name', 'circuit_country', 'circuit_name', 'driver_total_seasons'], axis=1)

df = df.dropna()
df

,id,driver_number,driver_season_rank,driver_season_points,driver_season_gps,driver_season_wins,driver_season_podiums,driver_season_poles,driver_season_fastlaps,driver_season_leadlaps,...,team_total_gps,team_seasons,circuit_length,circuit_temp,circuit_rain,quali_pos,quali_time,race_pos,circuit_id,team_id
0,1,1,1.0,364.0,15.0,12.0,14.0,8.0,6.0,661.0,...,719.0,19.0,5412.0,27.4317,0.0,1,89.708,1,7.0,9
1,2,11,2.0,219.0,15.0,2.0,8.0,2.0,2.0,138.0,...,719.0,19.0,5412.0,27.4317,0.0,2,89.846,2,7.0,9
2,3,14,3.0,170.0,15.0,0.0,7.0,0.0,1.0,3.0,...,126.0,5.0,5412.0,27.4317,0.0,5,90.336,3,7.0,4
3,4,55,5.0,117.0,15.0,0.0,1.0,1.0,0.0,14.0,...,2326.0,74.0,5412.0,27.4317,0.0,4,90.154,4,7.0,5
4,5,44,4.0,164.0,15.0,0.0,4.0,1.0,2.0,7.0,...,585.0,16.0,5412.0,27.4317,0.0,7,90.384,5,7.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159,1181,11,2.0,219.0,15.0,2.0,8.0,2.0,2.0,138.0,...,719.0,19.0,5281.0,24.5191,0.0,4,82.947,15,24.0,9
1160,1182,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1569.0,46.0,5281.0,24.5191,0.0,16,84.338,16,24.0,10
1161,1183,99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5281.0,24.5191,0.0,14,84.251,17,24.0,1
1162,1184,63,7.0,109.0,15.0,0.0,1.0,0.0,1.0,6.0,...,1569.0,46.0,5281.0,24.5191,0.0,17,84.423,18,24.0,10


In [344]:
data = df # <-- De este data set se extrae la info numérica

# Modelo: Random Forest Regressor

In [345]:
# Acá se crea y entrena el modelo

X = data[['driver_season_rank', 'driver_season_points', 'driver_season_gps',
          'driver_season_wins', 'driver_season_podiums', 'driver_season_poles', 'driver_season_fastlaps',
          'driver_season_leadlaps', 'driver_career_wins_avg', 'driver_career_podiums_avg',
          'driver_career_poles', 'driver_career_fastlaps', 'driver_career_gps', 'driver_career_season',
          'team_season_wins', 'team_season_podiums', 'team_season_poles', 'team_season_fastlaps',
          'team_season_leadlaps', 'team_season_points', 'team_average_wins', 'team_average_podiums',
          'team_total_gps', 'team_seasons', 'circuit_length', 'circuit_temp', 'circuit_rain', 'quali_pos',
          'quali_time', 'circuit_id', 'team_id']]
y = data['race_pos']

In [346]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [347]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 42)
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [348]:
y_pred = regressor.predict(X_test)

In [349]:
print(X_test.head(3))

print("Prediction: ", y_pred[0], '/', y_pred[1], '/', y_pred[2])

      driver_season_rank  driver_season_points  driver_season_gps  \
309                 14.0                   9.0               15.0   
1081                 0.0                   0.0                0.0   
381                  3.0                 170.0               15.0   

      driver_season_wins  driver_season_podiums  driver_season_poles  \
309                  0.0                    0.0                  0.0   
1081                 0.0                    0.0                  0.0   
381                  0.0                    7.0                  0.0   

      driver_season_fastlaps  driver_season_leadlaps  driver_career_wins_avg  \
309                      0.0                     0.0                    0.00   
1081                     0.0                     0.0                    0.00   
381                      1.0                     3.0                    0.09   

      driver_career_podiums_avg  ...  team_average_podiums  team_total_gps  \
309                        0.00  ..

In [350]:
# Predictions
# Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

# Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# R-squared (coefficient of determination)
r2 = r2_score(y_test, y_pred)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

print('Mean Squared Error: ', mse, '\n'
      'Root Mean Squared Error: ', rmse, '\n'
      'R-Squared: ', r2, '\n'
      'Mean Absolute Error: ', mae
      )

Mean Squared Error:  22.602453846153843 
Root Mean Squared Error:  4.754203807805661 
R-Squared:  0.3674433679123439 
Mean Absolute Error:  3.466651583710407


## Funciones

In [351]:
# 'driver_season_rank',
def get_driver_season_rank(id):
    
    query = """SELECT driver_season_rank from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res 

# 'driver_season_points',
def get_driver_season_points(id):
    
    query = """SELECT driver_season_points from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)

    return res

# 'driver_season_gps',
def get_driver_season_gps(id):
    
    query = """SELECT driver_season_gps from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)

    return res

# 'driver_season_wins',
def get_driver_season_wins(id):
    
    query = """SELECT driver_season_wins from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
     
    return res


# 'driver_season_podiums',
def get_driver_season_podiums(id):
    
    query = """SELECT driver_season_podiums from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
     
    return res

# 'driver_season_poles',
def get_driver_season_poles(id):
    
    query = """SELECT driver_season_poles from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
     
    return res

# 'driver_season_fastlaps',
def get_driver_season_fastlaps(id):
    
    query = """SELECT driver_season_fastlaps from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
     
    return res

# 'driver_season_leadlaps',
def get_driver_season_leadlaps(id):
    
    query = """SELECT driver_season_leadlaps from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
     
    return res

# 'driver_career_wins_avg',
def get_driver_career_wins(id):
    
    query = """SELECT driver_career_wins_avg from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1  """
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = (float(numRes))/100

    return res


# 'driver_career_podiums_avg',
def get_driver_career_podiums(id):
    
    query = """SELECT driver_career_podiums_avg from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = (float(numRes))/100
    
    return res


# 'driver_career_poles',
def get_driver_career_poles(id):
    
    query = """SELECT driver_career_poles from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'driver_career_fastlaps',
def get_driver_career_fastlaps(id):
    
    query = """SELECT driver_career_fastlaps from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'driver_career_gps',
def get_driver_career_gps(id):
    
    query = """SELECT driver_career_gps from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'driver_career_season',
def get_driver_career_season(id):
    
    query = """SELECT driver_career_season from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

#-------------------- Equipos --------------------

# 'team_id'
def get_team_id(id):
    
    query = """SELECT team_id from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = numRes
    
    return res

# 'team_season_wins',
def get_team_season_wins(id):
    
    query = """SELECT team_season_wins from f1_data WHERE team_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'team_season_podiums',
def get_team_season_podiums(id):
    
    query = """SELECT team_season_podiums from f1_data WHERE team_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'team_season_poles',
def get_team_season_poles(id):
    
    query = """SELECT team_season_poles from f1_data WHERE team_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'team_season_fastlaps',
def get_team_season_fastlaps(id):
    
    query = """SELECT team_season_fastlaps from f1_data WHERE team_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'team_season_leadlaps',
def get_team_season_leadlaps(id):
    
    query = """SELECT team_season_leadlaps from f1_data WHERE team_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'team_season_points',
def get_team_season_points(id):
    
    query = """SELECT team_season_points from f1_data WHERE team_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'team_average_wins',
def get_team_average_wins(id):
    
    query = """SELECT team_average_wins from f1_data WHERE team_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = (float(numRes))/100
    
    return res

# 'team_average_podiums',
def get_team_average_podiums(id):
    
    query = """SELECT team_average_podiums from f1_data WHERE team_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = (float(numRes))/100
    
    return res

# 'team_total_gps',
def get_team_total_gps(id):
    
    query = """SELECT team_total_gps from f1_data WHERE team_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'team_seasons',
def get_team_seasons(id):
    
    query = """SELECT team_seasons from f1_data WHERE team_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'circuit_length',
def get_circuit_length(id):
    
    query = """SELECT circuit_length from f1_data WHERE circuit_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

# 'circuit_temp',
def get_circuit_temp(id):
    
    query = """SELECT AVG(circuit_temp) from f1_data WHERE circuit_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)
    if numRes != '':
        numRes = (float(numRes))/1000000000000000
    else:
        numRes = 0

    if numRes < 10:
        numRes = numRes*10
    elif numRes > 100:
        numRes = numRes/10
    
    res = float("{:.4f}".format(numRes))
    
    return res

# 'circuit_rain',
def get_circuit_rain(id):
    
    query = """SELECT circuit_rain from f1_data WHERE circuit_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    if numRes != '':
        numRes = (float(numRes))/1000000
    else:
        numRes = 0
    
    res = float("{:.4f}".format(numRes))
    
    return res

# 'quali_pos',
def get_quali_pos(id, race):

    query = """SELECT AVG(quali_pos) from f1_data WHERE driver_number = %s AND circuit_id = %s"""
    cursor.execute(query, (id, race))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)
    if numRes != '':
        numRes = (float(numRes))/10000
    else:
        numRes = 0

    return numRes

# 'quali_time',
def get_quali_time(id, race):
    
    query = """SELECT AVG(quali_time) from f1_data WHERE driver_number = %s AND circuit_id = %s"""
    cursor.execute(query, (id, race))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)
    if numRes != '':
        numRes = (float(numRes))/100000000000000
    else:
        numRes = 0

    if numRes < 60:
        numRes = numRes*10
    
    res = float("{:.3f}".format(numRes))

    return res

# 'circuit_id',
def get_circuit_id(id):
    
    query = """SELECT circuit_id from f1_data WHERE circuit_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    numRes = re.sub('\D', '', resStr)

    res = int(numRes)
    
    return res

def get_team_name(id):

    query = """SELECT team_name from f1_data WHERE team_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    resStr1 = resStr.replace("('", "")
    resStr1 = resStr1.replace("',)", "")

    return resStr1

#-----------------
# Get driver name
def get_driver_name(id):

    query = """SELECT driver_name from f1_data WHERE driver_number = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    resStr1 = resStr.replace("('", "")
    resStr1 = resStr1.replace("',)", "")

    return resStr1

def get_circuit_name(id):

    query = """SELECT circuit_name from f1_data WHERE circuit_id = %s ORDER BY driver_number desc limit 1"""
    cursor.execute(query, (id,))
    res = cursor.fetchall()

    resStr = ''.join(map(str, res))

    resStr1 = resStr.replace("('", "")
    resStr1 = resStr1.replace("',)", "")

    return resStr1

In [443]:
numD = int(input("Ingrese el número del piloto"))
# numR = int(input("Ingrese el código de la carrera"))
numR = 18

In [444]:
a = get_driver_season_rank(numD)
print(f'Rango del piloto: {a}')

b = get_driver_season_points(numD)
print(f'Puntos del piloto: {b}')
 
c = get_driver_season_gps(numD)
print(f'GrandPrix del piloto esta temporada: {c}')

d = get_driver_season_wins(numD)
print(f'Victorias del piloto esta temporada: {d}')

e = get_driver_season_podiums(numD)
print(f'Podios del piloto este temporada: {e}')
 
f = get_driver_season_poles(numD)
print(f'Poles del piloto esta temporada: {f}')
 
g = get_driver_season_fastlaps(numD)
print(f'Vueltas rápidas del piloto esta temporada: {g}')
 
h = get_driver_season_leadlaps(numD)
print(f'Vueltas lideradas del piloto esta temporada: {h}')
 
i = get_driver_career_wins(numD)
print(f'Victorias totales del piloto : {i}')
 
j = get_driver_career_podiums(numD)
print(f'Podios totales del piloto : {j}')

k = get_driver_career_poles(numD)
print(f'Poles totales del piloto : {k}')
 
l = get_driver_career_fastlaps(numD)
print(f'Vueltas rápidas totales del piloto : {l}')
 
m = get_driver_career_gps(numD)
print(f'GrandPrix totales del piloto : {m}')
 
n = get_driver_career_season(numD)
print(f'Temporadas del piloto en F1 : {n}')


#-------------------- Equipos --------------------

aa = get_team_id(numD)
print(f'Id del equipo : {aa}')

ff = get_team_name(aa)
print(f'Nombre del equipo : {ff}')
 
o = get_team_season_wins(aa)
print(f'Victorias del equipo esta temporada: {o}')
 
p = get_team_season_podiums(aa)
print(f'Podios del equipo esta temporada: {p}')
 
q = get_team_season_poles(aa)
print(f'Poles del equipo esta temporada: {q}')
 
r = get_team_season_fastlaps(aa)
print(f'Vueltas rápidas del equipo esta temporada: {r}')
 
s = get_team_season_leadlaps(aa)
print(f'Vueltas lideradas del equipo esta temporada: {s}')
 
t = get_team_season_points(aa)
print(f'Puntos del equipo esta temporada: {t}')
 
u = get_team_average_wins(aa)
print(f'Victorias /GP del equipo en su historia: {u}')
 
v = get_team_average_podiums(aa)
print(f'Podios /GP del equipo en su historia: {v}')
 
w = get_team_total_gps(aa)
print(f'GP del equipo en su historia: {w}')
 
xx = get_team_seasons(aa)
print(f'Temporadas del equipo en F1: {xx}')
 
#-------------------- Pista --------------------

hh = get_circuit_name(numR)
print(f'Nombre del circuito: {hh}')
 
yy = get_circuit_length(numR)
print(f'Longuitud del circuito: {yy}')
 
z = get_circuit_temp(numR)
print(f'Temperatura del circuito: {z}')
 
bb = get_circuit_rain(numR)
print(f'Lluvia del circuito: {bb}')
 

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
cc = get_quali_pos(numD, numR)
print(f'Posición promedio del piloto circuito: {cc}')
 
dd = get_quali_time(numD, numR)
print(f'Tiempo de quali del piloto del circuito: {dd}')
 
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# ee = get_circuit_id(numR)

gg = get_driver_name(numD)
print(f'\n DATOS PARA EL PILOTO: {gg}')

Rango del piloto: 20
Puntos del piloto: 0
GrandPrix del piloto esta temporada: 2
Victorias del piloto esta temporada: 0
Podios del piloto este temporada: 0
Poles del piloto esta temporada: 0
Vueltas rápidas del piloto esta temporada: 0
Vueltas lideradas del piloto esta temporada: 0
Victorias totales del piloto : 0.0
Podios totales del piloto : 0.0
Poles totales del piloto : 0
Vueltas rápidas totales del piloto : 0
GrandPrix totales del piloto : 2
Temporadas del piloto en F1 : 1
Id del equipo : 10
Nombre del equipo : AlphaTauri
Victorias del equipo esta temporada: 0
Podios del equipo esta temporada: 0
Poles del equipo esta temporada: 0
Vueltas rápidas del equipo esta temporada: 0
Vueltas lideradas del equipo esta temporada: 0
Puntos del equipo esta temporada: 3
Victorias /GP del equipo en su historia: 0.0
Podios /GP del equipo en su historia: 0.01
GP del equipo en su historia: 684
Temporadas del equipo en F1: 18
Nombre del circuito: Marina Bay Street Circuit
Longuitud del circuito: 5065

In [445]:
driverSeasonRank = get_driver_season_rank(numD)
driverSeasonPoints = get_driver_season_points(numD)
driverSeasonGps = get_driver_season_gps(numD)
driverSeasonWins = get_driver_season_wins(numD)
driverSeasonPodiums = get_driver_season_podiums(numD)
driverSeasonPoles = get_driver_season_poles(numD)
driverSeasonFastLaps = get_driver_season_fastlaps(numD)
driverSeasonLeadlaps = get_driver_season_leadlaps(numD)
driverCareerWins = get_driver_career_wins(numD)
driverCareerPodiums = get_driver_career_podiums(numD)
driverCareerPoles = get_driver_career_poles(numD)
driverCareerFastlaps = get_driver_career_fastlaps(numD)
driverCareerGps = get_driver_career_gps(numD)
driverCareerSeason = get_driver_career_season(numD)

#-------------------------- Equipos --------------------------
#Estos dos no entran en el data set
teamId = get_team_id(numD)
teamName = get_team_name(teamId)
 
teamSeasonWins = get_team_season_wins(teamId)
teamSeasonPodiums = get_team_season_podiums(teamId)
teamSeasonPoles = get_team_season_poles(teamId)
teamSeasonFastlaps = get_team_season_fastlaps(teamId)
teamSeasonLeadlaps = get_team_season_leadlaps(teamId)
teamSeasonPoints = get_team_season_points(teamId)
teamAverageWins = get_team_average_wins(teamId)
teamAveragePodiums = get_team_average_podiums(teamId)
teamTotalGps = get_team_total_gps(teamId)
teamSeasons = get_team_seasons(teamId)

#-------------------------- Pista --------------------------

#Este no va
circuitName = get_circuit_name(numR)

circuitLength = get_circuit_length(numR)
circuitTemp = get_circuit_temp(numR)
circuitRain = get_circuit_rain(numR)

qualiPos = get_quali_pos(numD, numR)
qualiTime = get_quali_time(numD, numR)

driverName = get_driver_name(numD)

## Test

### Crear modelo a enviar

In [446]:
datos = {'driver_season_rank': [driverSeasonRank],
         'driver_season_points': [driverSeasonPoints],
         'driver_season_gps': [driverSeasonGps],
         'driver_season_wins': [driverSeasonWins],
         'driver_season_podiums': [driverSeasonPodiums],
         'driver_season_poles': [driverSeasonPoles],
         'driver_season_fastlaps': [driverSeasonFastLaps],
         'driver_season_leadlaps': [driverSeasonLeadlaps],
         'driver_career_wins_avg': [driverCareerWins],
         'driver_career_podiums_avg': [driverCareerPodiums], #10
         'driver_career_poles': [driverCareerPoles],
         'driver_career_fastlaps': [driverCareerFastlaps],
         'driver_career_gps': [driverCareerGps],
         'driver_career_season': [driverCareerSeason],
         'team_season_wins': [teamSeasonWins],
         'team_season_podiums': [teamSeasonPodiums],
         'team_season_poles': [teamSeasonPoles], #17
         'team_season_fastlaps': [teamSeasonFastlaps],
         'team_season_leadlaps': [teamSeasonLeadlaps],
         'team_season_points': [teamSeasonPoints], #20
         'team_average_wins': [teamAverageWins],
         'team_average_podiums': [teamAveragePodiums],
         'team_total_gps': [teamTotalGps],
         'team_seasons': [teamSeasons],
         'circuit_length': [circuitLength],
         'circuit_temp': [circuitTemp],
         'circuit_rain': [circuitRain],
         'quali_pos': [qualiPos],
         'quali_time': [qualiTime],
         'circuit_id': [numR], #30
         'team_id': [teamId]
         }

In [447]:
dfEnviar = pd.DataFrame(datos)
dfEnviar

,driver_season_rank,driver_season_points,driver_season_gps,driver_season_wins,driver_season_podiums,driver_season_poles,driver_season_fastlaps,driver_season_leadlaps,driver_career_wins_avg,driver_career_podiums_avg,...,team_average_podiums,team_total_gps,team_seasons,circuit_length,circuit_temp,circuit_rain,quali_pos,quali_time,circuit_id,team_id
0,20,0,2,0,0,0,0,0,0.0,0.0,...,0.01,684,18,5065,2.6233,0.1804,0,0.0,18,10


### Resultados

In [448]:
nueva_pred = regressor.predict(dfEnviar)

print(f'Para el piloto {driverName} en la pista {circuitName} la posición predecida es {nueva_pred}')


Para el piloto L LAWSON en la pista Marina Bay Street Circuit la posición predecida es [10.73]
